In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import json
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
import _pickle as cPickle
import os
import s3fs
import boto3
import math
import csv
import re

plt.rcdefaults()

# Blood data

- Comparison of top CpG sites for Parkinson's, Alzheimer's and Healthy

In [128]:
s3 = s3fs.S3FileSystem(anon=False)
bucket='ewasunhealthy'
data_key = 'data/blood/alzheimers/alz_blood_unhealthy.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)
alz_blood_unhealthy = pd.read_csv(data_location, index_col=0,low_memory=False)

In [129]:
alz_blood_unhealthy

,AGE,cg00050873,cg00212031,cg00213748,cg00214611,cg00455876,cg01707559,cg02004872,cg02011394,cg02050847,...,cg04103088,cg04105342,cg04106633,cg04107037,cg04111054,cg04112173,cg04112590,cg04112626,cg04113154,cg04114386
GSM1069225,81.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.224,0.951,0.050,NaN,0.965,NaN,NaN,0.991,NaN
GSM1069226,88.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.197,0.837,0.040,NaN,0.986,NaN,NaN,0.984,NaN
GSM1069227,91.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.979,0.120,0.957,0.024,NaN,0.982,NaN,NaN,0.989,NaN
GSM1069228,87.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.241,0.863,0.091,NaN,0.980,NaN,0.275,0.992,NaN
GSM1069229,89.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.993,0.110,0.947,0.024,NaN,0.991,NaN,NaN,0.989,0.963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM1443699,88.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.988,0.164,0.715,0.031,NaN,0.990,NaN,0.157,0.982,0.938
GSM1443711,74.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.986,0.213,0.940,0.024,NaN,0.965,NaN,NaN,0.992,NaN
GSM1443721,74.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.976,0.256,0.934,0.022,NaN,0.976,NaN,0.184,0.985,0.964
GSM1443736,89.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.972,0.182,0.945,0.051,NaN,0.967,NaN,NaN,0.988,0.965


In [130]:
park_blood_unhealthy = pd.read_csv('s3://ewasunhealthy/data/blood/parkinsons/park_blood_unhealthy.csv', index_col=0,low_memory=False)

In [131]:
park_blood_unhealthy

,AGE,cg00050873,cg00212031,cg00213748,cg00214611,cg00455876,cg01707559,cg02004872,cg02011394,cg02050847,...,cg04103088,cg04105342,cg04106633,cg04107037,cg04111054,cg04112173,cg04112590,cg04112626,cg04113154,cg04114386
GSM3035963,78.0,0.925,NaN,NaN,NaN,NaN,0.144,0.026,0.966,0.963,...,0.977,0.061,0.949,0.016,NaN,0.979,NaN,0.056,0.988,NaN
GSM3035809,78.0,0.863,0.118,NaN,0.032,0.840,0.166,0.019,0.966,0.991,...,0.990,0.285,0.907,0.032,NaN,0.974,0.874,0.119,0.992,0.944
GSM3035847,75.0,0.893,0.038,NaN,NaN,0.738,0.118,0.013,0.971,0.994,...,NaN,0.235,0.864,0.012,NaN,0.979,NaN,0.077,0.989,NaN
GSM1870906,71.4,0.897,0.014,NaN,0.035,0.778,0.112,0.024,0.962,0.993,...,0.983,0.161,0.920,0.042,NaN,0.988,NaN,0.104,0.995,0.953
GSM1871363,76.2,NaN,NaN,NaN,NaN,NaN,0.173,NaN,NaN,NaN,...,0.986,0.259,0.962,0.038,NaN,0.967,NaN,0.111,0.987,0.965
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM3035602.1,79.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.289,0.948,0.049,NaN,0.975,NaN,0.133,0.987,NaN
GSM1343558.1,72.2,0.816,0.036,NaN,0.020,0.793,0.108,0.024,0.974,0.987,...,0.991,0.225,0.847,0.055,NaN,0.981,NaN,0.148,0.969,0.980
GSM1870847.1,76.3,NaN,NaN,NaN,NaN,NaN,0.283,NaN,NaN,NaN,...,0.996,0.237,0.896,0.027,NaN,0.983,0.857,0.091,0.986,0.979
GSM1498556.1,62.0,NaN,NaN,NaN,NaN,NaN,0.216,NaN,NaN,NaN,...,NaN,0.256,0.966,0.038,NaN,0.986,NaN,NaN,0.977,NaN


In [9]:
with s3.open('s3://ewasunhealthy/data/blood/alzheimers/xgb_alz_blood_cpgs', "rb") as input_file:
    alz_blood_top_cpgs = cPickle.load(input_file)
xgb_alz_blood_cpgs=list(alz_blood_top_cpgs)


In [10]:
with s3.open('s3://ewasunhealthy/data/blood/parkinsons/xgb_park_blood_cpgs', "rb") as input_file:
    park_blood_top_cpgs = cPickle.load(input_file)
xgb_park_blood_cpgs=list(park_blood_top_cpgs)

# Healthy

In [132]:
with open(r"cpgs_XGboost_whole_blood_ranked", "rb") as input_file:
    xgb_hc_blood_cpgs = cPickle.load(input_file)
xgb_hc_blood_cpgs=list(xgb_hc_blood_cpgs)

In [12]:
print('Features in top 100 for all 3')
for i in xgb_hc_blood_cpgs[0:100]:    
    if i in xgb_park_blood_cpgs[0:100] and i in xgb_alz_blood_cpgs[0:100]:       
        print('|',i,' Healthy = ',xgb_hc_blood_cpgs.index(i),' Alz = ',xgb_alz_blood_cpgs.index(i),' Park = ',xgb_park_blood_cpgs.index(i),'|')


Features in top 100 for all 3
| cg16867657  Healthy =  1  Alz =  24  Park =  1 |
| cg04875128  Healthy =  2  Alz =  2  Park =  4 |
| cg07553761  Healthy =  4  Alz =  8  Park =  8 |
| cg22454769  Healthy =  9  Alz =  50  Park =  11 |


In [13]:
print('Features in top 100 for Alzheimers')
for i in xgb_alz_blood_cpgs[0:100]:    
    if i in xgb_park_blood_cpgs and i in xgb_hc_blood_cpgs:       
        print('|',i,' Healthy = ',xgb_hc_blood_cpgs.index(i),' Alz = ',xgb_alz_blood_cpgs.index(i),' Park = ',xgb_park_blood_cpgs.index(i),'|')


Features in top 100 for Alzheimers
| cg04875128  Healthy =  2  Alz =  2  Park =  4 |
| cg16290275  Healthy =  1268  Alz =  6  Park =  4573 |
| cg07553761  Healthy =  4  Alz =  8  Park =  8 |
| cg16867657  Healthy =  1  Alz =  24  Park =  1 |
| cg19381766  Healthy =  775  Alz =  33  Park =  256 |
| cg26671246  Healthy =  2205  Alz =  41  Park =  147512 |
| cg16008966  Healthy =  111  Alz =  43  Park =  20760 |
| cg22454769  Healthy =  9  Alz =  50  Park =  11 |
| cg01074797  Healthy =  63  Alz =  56  Park =  963 |


In [14]:
print('Features in top 100 for Parkinsons')
for i in xgb_park_blood_cpgs[0:100]:    
    if i in xgb_alz_blood_cpgs and i in xgb_hc_blood_cpgs:       
        print('|',i,' Healthy = ',xgb_hc_blood_cpgs.index(i),' Alz = ',xgb_alz_blood_cpgs.index(i),' Park = ',xgb_park_blood_cpgs.index(i),'|')


Features in top 100 for Parkinsons
| cg15121420  Healthy =  145  Alz =  166916  Park =  0 |
| cg16867657  Healthy =  1  Alz =  24  Park =  1 |
| cg23744638  Healthy =  20  Alz =  217001  Park =  2 |
| cg16932827  Healthy =  31  Alz =  86433  Park =  3 |
| cg04875128  Healthy =  2  Alz =  2  Park =  4 |
| cg07181952  Healthy =  98  Alz =  153181  Park =  5 |
| cg08160331  Healthy =  1883  Alz =  221999  Park =  6 |
| cg24724428  Healthy =  23  Alz =  131380  Park =  7 |
| cg07553761  Healthy =  4  Alz =  8  Park =  8 |
| cg07955995  Healthy =  62  Alz =  134754  Park =  9 |
| cg22158769  Healthy =  176  Alz =  157884  Park =  10 |
| cg22454769  Healthy =  9  Alz =  50  Park =  11 |
| cg06784991  Healthy =  38  Alz =  155297  Park =  12 |
| cg06639320  Healthy =  10  Alz =  1307  Park =  14 |
| cg23078123  Healthy =  28  Alz =  99458  Park =  15 |
| cg27619353  Healthy =  729  Alz =  56469  Park =  16 |
| cg24079702  Healthy =  51  Alz =  158767  Park =  17 |
| cg23606718  Healthy =  18 

In [15]:
comb=[]
for i in xgb_hc_blood_cpgs[0:100]:    
    if i in xgb_alz_blood_cpgs and i in xgb_park_blood_cpgs and i not in comb:       
        comb.append(i)

for i in xgb_alz_blood_cpgs[0:100]:    
    if i in xgb_park_blood_cpgs and i in xgb_hc_blood_cpgs and i not in comb:       
        comb.append(i)
        
for i in xgb_park_blood_cpgs[0:100]:    
    if i in xgb_alz_blood_cpgs and i in xgb_hc_blood_cpgs and i not in comb:       
        comb.append(i)

In [16]:
for i in comb:
    print('|',i,' Healthy = ',xgb_hc_blood_cpgs.index(i),' Alz = ',xgb_alz_blood_cpgs.index(i),' Park = ',xgb_park_blood_cpgs.index(i),'|')

| cg16867657  Healthy =  1  Alz =  24  Park =  1 |
| cg04875128  Healthy =  2  Alz =  2  Park =  4 |
| cg07553761  Healthy =  4  Alz =  8  Park =  8 |
| cg00292135  Healthy =  5  Alz =  3401  Park =  5257 |
| cg09809672  Healthy =  6  Alz =  24844  Park =  126 |
| cg04208403  Healthy =  7  Alz =  195727  Park =  1963 |
| cg22454769  Healthy =  9  Alz =  50  Park =  11 |
| cg06639320  Healthy =  10  Alz =  1307  Park =  14 |
| cg19344626  Healthy =  13  Alz =  942  Park =  125 |
| cg02046143  Healthy =  15  Alz =  224091  Park =  84 |
| cg23606718  Healthy =  18  Alz =  158644  Park =  18 |
| cg08453194  Healthy =  19  Alz =  135399  Park =  36 |
| cg23744638  Healthy =  20  Alz =  217001  Park =  2 |
| cg07927379  Healthy =  21  Alz =  4437  Park =  306 |
| cg24724428  Healthy =  23  Alz =  131380  Park =  7 |
| cg23078123  Healthy =  28  Alz =  99458  Park =  15 |
| cg16932827  Healthy =  31  Alz =  86433  Park =  3 |
| cg10221746  Healthy =  33  Alz =  25470  Park =  316 |
| cg188773

In [17]:
len(comb)

77

In [18]:
def read_by_cpg(df_filename, cpg_list):
    allcpgs = []
    with open(df_filename) as f:
        for row in f:
            if row[0:3]!='ch.':
                allcpgs.append(row[0:10])
            else:
                allcpgs.append(row[0:14])
    # we want to skip the rows that are not in cpg_list
    skipped = []
    for i in range(1,len(allcpgs)):
        if allcpgs[i] not in cpg_list:
            skipped.append(i)
            
    df = pd.read_csv(df_filename, skiprows=skipped,low_memory=False)
    return df

In [19]:
import s3fs
import boto3
import math
import csv
import re

s3 = s3fs.S3FileSystem(anon=False)


In [20]:
bucket='age-data-reuploaded'
data_key = 'whole_blood_age_methylation_v1.txt'
data_location = 's3://{}/{}'.format(bucket, data_key)

rows_hc=[]
with s3.open(data_location, 'r') as file_in:
    for row in file_in:
        rows_hc.append(row[0:10])

In [21]:
skipped = []
for i in range(1,len(rows_hc)):
    if rows_hc[i] not in comb:
        skipped.append(i)

In [22]:
cl = pd.read_csv(data_location, skiprows=skipped, low_memory=False)
cl

,sample_id,GSM2334366,GSM989882,GSM989863,GSM1443696,GSM1069241,GSM1572402,GSM1572442,GSM1648928,GSM990558,...,GSM2815698,GSM2814296,GSM2814963,GSM2814230,GSM2814121,GSM2815345,GSM2814886,GSM2815335,GSM2334524,GSM2815609
0,cg01074797,0.688,0.504,0.571,0.440,0.440,0.539,0.599,0.522,0.530,...,0.745,0.751,0.712,0.094,0.783,0.785,0.701,0.730,0.678,0.767
1,cg06784991,0.457,0.316,0.435,0.335,0.335,0.314,0.287,0.275,0.332,...,0.262,0.235,0.224,0.233,0.211,0.199,0.240,0.224,0.190,0.178
2,cg07181952,0.688,0.602,0.673,0.636,0.636,0.553,0.570,0.667,0.597,...,0.770,0.730,0.743,0.768,0.760,0.699,0.673,0.717,0.699,0.690
3,cg07418114,0.713,0.664,0.634,0.663,0.663,0.591,0.593,0.660,0.608,...,0.750,0.656,0.723,0.779,0.748,0.702,0.719,0.716,0.794,0.715
4,cg16290275,0.232,0.193,0.148,0.143,0.143,0.196,0.204,0.374,0.101,...,0.314,0.256,0.349,0.349,0.318,0.272,0.307,0.358,0.349,0.289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,cg00329615,0.357,0.377,0.327,0.385,0.385,0.389,0.329,0.304,0.177,...,0.656,0.645,0.622,0.636,0.684,0.598,0.634,0.650,0.731,0.605
73,cg03607117,0.194,0.078,0.096,0.180,0.180,0.078,0.053,0.202,0.106,...,0.026,0.030,0.025,0.059,0.038,0.040,0.033,0.021,0.051,0.027
74,cg04474832,0.300,0.223,0.190,0.232,0.232,0.225,0.190,0.236,0.139,...,0.328,0.287,0.335,0.260,0.305,0.279,0.338,0.292,0.311,0.318
75,cg16932827,0.835,0.659,0.650,0.611,0.611,0.653,0.566,0.753,0.666,...,0.959,0.920,0.967,0.956,0.940,0.886,0.939,0.889,0.951,0.948


In [23]:
#meta = pd.read_csv('sample_disease.txt', delimiter = '\t')
meta = []

with open(r'sample_age.txt') as f:
    for row in f:
        meta.append(row[0:50])

meta_2 =[]

for i in meta:
    meta_2.append(i.split())

ages=[]
df_id_ages=[]
df_age=[]
for i in range(1,len(meta_2)):
    p=meta_2[i]
    
    v=[]
    if p[0][1]=='G':
        v.append(p[0][1:-1])
        v.append(p[2])
        df_id_ages.append(p[0][1:-1])
        df_age.append(p[2])
        ages.append(v)
    
    
df_ages = pd.DataFrame({'id': df_id_ages,
                   'age': df_age,})

df_ages=df_ages.set_index('id')
df_ages

,age
id,
GSM2139432,71
GSM2139249,76
GSM2139398,102
GSM2139297,108
GSM1069208,40
...,...
GSM1871753,53
GSM1438255,53.1594
GSM1438138,53.3855


In [24]:
healthy_df=cl.set_index('sample_id').T


In [25]:
healthy_df

sample_id,cg01074797,cg06784991,cg07181952,cg07418114,cg16290275,cg26671246,cg15121420,cg22158769,cg22454769,cg23606718,...,cg23078123,cg25256723,cg25410668,cg06240854,cg06639320,cg00329615,cg03607117,cg04474832,cg16932827,cg26614073
GSM2334366,0.688,0.457,0.688,0.713,0.232,0.598,0.565,0.096,0.770,0.294,...,0.465,0.805,0.597,0.474,0.594,0.357,0.194,0.300,0.835,0.287
GSM989882,0.504,0.316,0.602,0.664,0.193,0.582,0.429,0.070,0.857,0.269,...,0.558,0.576,0.699,0.391,0.704,0.377,0.078,0.223,0.659,0.592
GSM989863,0.571,0.435,0.673,0.634,0.148,0.557,0.442,0.047,0.751,0.151,...,0.471,0.554,0.652,0.236,0.619,0.327,0.096,0.190,0.650,0.506
GSM1443696,0.440,0.335,0.636,0.663,0.143,0.557,0.537,0.110,0.723,0.254,...,0.560,0.714,0.663,0.537,0.579,0.385,0.180,0.232,0.611,0.422
GSM1069241,0.440,0.335,0.636,0.663,0.143,0.557,0.537,0.110,0.723,0.254,...,0.560,0.714,0.663,0.537,0.579,0.385,0.180,0.232,0.611,0.422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM2815345,0.785,0.199,0.699,0.702,0.272,0.655,0.600,0.015,0.312,0.057,...,0.693,0.822,0.468,0.533,0.292,0.598,0.040,0.279,0.886,0.547
GSM2814886,0.701,0.240,0.673,0.719,0.307,0.640,0.642,0.013,0.347,0.097,...,0.706,0.763,0.557,0.516,0.381,0.634,0.033,0.338,0.939,0.577
GSM2815335,0.730,0.224,0.717,0.716,0.358,0.639,0.643,0.009,0.327,0.074,...,0.754,0.785,0.513,0.599,0.347,0.650,0.021,0.292,0.889,0.617
GSM2334524,0.678,0.190,0.699,0.794,0.349,0.678,0.633,0.005,0.415,0.080,...,0.741,0.799,0.520,0.533,0.338,0.731,0.051,0.311,0.951,0.530


In [26]:
healthy_df['AGE']=np.zeros(len(healthy_df.index))
cols = list(healthy_df.columns.values)
cols=cols[0:-1]
healthy_df = healthy_df[['AGE']+cols]
healthy_df

sample_id,AGE,cg01074797,cg06784991,cg07181952,cg07418114,cg16290275,cg26671246,cg15121420,cg22158769,cg22454769,...,cg23078123,cg25256723,cg25410668,cg06240854,cg06639320,cg00329615,cg03607117,cg04474832,cg16932827,cg26614073
GSM2334366,0.0,0.688,0.457,0.688,0.713,0.232,0.598,0.565,0.096,0.770,...,0.465,0.805,0.597,0.474,0.594,0.357,0.194,0.300,0.835,0.287
GSM989882,0.0,0.504,0.316,0.602,0.664,0.193,0.582,0.429,0.070,0.857,...,0.558,0.576,0.699,0.391,0.704,0.377,0.078,0.223,0.659,0.592
GSM989863,0.0,0.571,0.435,0.673,0.634,0.148,0.557,0.442,0.047,0.751,...,0.471,0.554,0.652,0.236,0.619,0.327,0.096,0.190,0.650,0.506
GSM1443696,0.0,0.440,0.335,0.636,0.663,0.143,0.557,0.537,0.110,0.723,...,0.560,0.714,0.663,0.537,0.579,0.385,0.180,0.232,0.611,0.422
GSM1069241,0.0,0.440,0.335,0.636,0.663,0.143,0.557,0.537,0.110,0.723,...,0.560,0.714,0.663,0.537,0.579,0.385,0.180,0.232,0.611,0.422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM2815345,0.0,0.785,0.199,0.699,0.702,0.272,0.655,0.600,0.015,0.312,...,0.693,0.822,0.468,0.533,0.292,0.598,0.040,0.279,0.886,0.547
GSM2814886,0.0,0.701,0.240,0.673,0.719,0.307,0.640,0.642,0.013,0.347,...,0.706,0.763,0.557,0.516,0.381,0.634,0.033,0.338,0.939,0.577
GSM2815335,0.0,0.730,0.224,0.717,0.716,0.358,0.639,0.643,0.009,0.327,...,0.754,0.785,0.513,0.599,0.347,0.650,0.021,0.292,0.889,0.617
GSM2334524,0.0,0.678,0.190,0.699,0.794,0.349,0.678,0.633,0.005,0.415,...,0.741,0.799,0.520,0.533,0.338,0.731,0.051,0.311,0.951,0.530


In [27]:
for i in healthy_df.index:
    try:
        healthy_df.loc[i,'AGE']=float(df_ages.loc[i]['age'])
    except KeyError:
        healthy_df.loc[i,'AGE']='NaN'
healthy_df


sample_id,AGE,cg01074797,cg06784991,cg07181952,cg07418114,cg16290275,cg26671246,cg15121420,cg22158769,cg22454769,...,cg23078123,cg25256723,cg25410668,cg06240854,cg06639320,cg00329615,cg03607117,cg04474832,cg16932827,cg26614073
GSM2334366,94.0,0.688,0.457,0.688,0.713,0.232,0.598,0.565,0.096,0.770,...,0.465,0.805,0.597,0.474,0.594,0.357,0.194,0.300,0.835,0.287
GSM989882,96.0,0.504,0.316,0.602,0.664,0.193,0.582,0.429,0.070,0.857,...,0.558,0.576,0.699,0.391,0.704,0.377,0.078,0.223,0.659,0.592
GSM989863,101.0,0.571,0.435,0.673,0.634,0.148,0.557,0.442,0.047,0.751,...,0.471,0.554,0.652,0.236,0.619,0.327,0.096,0.190,0.650,0.506
GSM1443696,99.0,0.440,0.335,0.636,0.663,0.143,0.557,0.537,0.110,0.723,...,0.560,0.714,0.663,0.537,0.579,0.385,0.180,0.232,0.611,0.422
GSM1069241,99.0,0.440,0.335,0.636,0.663,0.143,0.557,0.537,0.110,0.723,...,0.560,0.714,0.663,0.537,0.579,0.385,0.180,0.232,0.611,0.422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM2815345,18.0,0.785,0.199,0.699,0.702,0.272,0.655,0.600,0.015,0.312,...,0.693,0.822,0.468,0.533,0.292,0.598,0.040,0.279,0.886,0.547
GSM2814886,18.0,0.701,0.240,0.673,0.719,0.307,0.640,0.642,0.013,0.347,...,0.706,0.763,0.557,0.516,0.381,0.634,0.033,0.338,0.939,0.577
GSM2815335,18.0,0.730,0.224,0.717,0.716,0.358,0.639,0.643,0.009,0.327,...,0.754,0.785,0.513,0.599,0.347,0.650,0.021,0.292,0.889,0.617
GSM2334524,18.0,0.678,0.190,0.699,0.794,0.349,0.678,0.633,0.005,0.415,...,0.741,0.799,0.520,0.533,0.338,0.731,0.051,0.311,0.951,0.530


In [28]:
healthy_df['status']=[0]*len(healthy_df.index)
cols = list(healthy_df.columns.values)
cols=cols[0:-1]
healthy_df = healthy_df[['status']+cols]
healthy_df

sample_id,status,AGE,cg01074797,cg06784991,cg07181952,cg07418114,cg16290275,cg26671246,cg15121420,cg22158769,...,cg23078123,cg25256723,cg25410668,cg06240854,cg06639320,cg00329615,cg03607117,cg04474832,cg16932827,cg26614073
GSM2334366,0,94.0,0.688,0.457,0.688,0.713,0.232,0.598,0.565,0.096,...,0.465,0.805,0.597,0.474,0.594,0.357,0.194,0.300,0.835,0.287
GSM989882,0,96.0,0.504,0.316,0.602,0.664,0.193,0.582,0.429,0.070,...,0.558,0.576,0.699,0.391,0.704,0.377,0.078,0.223,0.659,0.592
GSM989863,0,101.0,0.571,0.435,0.673,0.634,0.148,0.557,0.442,0.047,...,0.471,0.554,0.652,0.236,0.619,0.327,0.096,0.190,0.650,0.506
GSM1443696,0,99.0,0.440,0.335,0.636,0.663,0.143,0.557,0.537,0.110,...,0.560,0.714,0.663,0.537,0.579,0.385,0.180,0.232,0.611,0.422
GSM1069241,0,99.0,0.440,0.335,0.636,0.663,0.143,0.557,0.537,0.110,...,0.560,0.714,0.663,0.537,0.579,0.385,0.180,0.232,0.611,0.422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM2815345,0,18.0,0.785,0.199,0.699,0.702,0.272,0.655,0.600,0.015,...,0.693,0.822,0.468,0.533,0.292,0.598,0.040,0.279,0.886,0.547
GSM2814886,0,18.0,0.701,0.240,0.673,0.719,0.307,0.640,0.642,0.013,...,0.706,0.763,0.557,0.516,0.381,0.634,0.033,0.338,0.939,0.577
GSM2815335,0,18.0,0.730,0.224,0.717,0.716,0.358,0.639,0.643,0.009,...,0.754,0.785,0.513,0.599,0.347,0.650,0.021,0.292,0.889,0.617
GSM2334524,0,18.0,0.678,0.190,0.699,0.794,0.349,0.678,0.633,0.005,...,0.741,0.799,0.520,0.533,0.338,0.731,0.051,0.311,0.951,0.530


In [ ]:
park_blood_unhealthy

In [31]:
col_p=list(park_blood_unhealthy.columns[1:])
drop_p=[]
for i in col_p:
    if i not in comb:
        drop_p.append(i)

park_blood_unhealthy=park_blood_unhealthy.drop(columns=drop_p)    
park_blood_unhealthy   

,AGE,cg01074797,cg06784991,cg07181952,cg07418114,cg16290275,cg26671246,cg15121420,cg22158769,cg22454769,...,cg23078123,cg25256723,cg25410668,cg06240854,cg06639320,cg00329615,cg03607117,cg04474832,cg16932827,cg26614073
GSM3035963,78.0,0.528,0.274,0.624,0.685,0.257,0.535,0.569,0.008,0.702,...,0.443,0.722,0.948,0.475,0.589,0.322,0.086,0.176,0.772,0.394
GSM3035809,78.0,0.581,0.333,0.696,0.643,0.208,0.597,0.521,0.029,0.606,...,0.556,0.713,0.744,0.365,0.532,0.439,0.088,0.232,0.775,0.372
GSM3035847,75.0,0.532,0.242,0.673,0.639,0.184,0.551,0.486,0.045,0.512,...,0.542,0.619,0.733,0.392,0.455,0.346,0.043,0.191,0.661,0.435
GSM1870906,71.4,0.555,0.321,0.665,0.627,0.230,0.580,0.479,0.024,0.717,...,0.591,0.638,0.917,0.409,0.576,0.250,0.081,0.215,0.661,0.417
GSM1871363,76.2,0.640,0.334,0.626,0.664,0.222,0.598,0.509,0.113,0.692,...,0.589,0.784,0.723,0.404,0.617,0.405,0.139,0.224,0.840,0.500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM3035602.1,79.0,0.595,0.289,0.593,0.689,0.227,0.574,0.547,0.042,0.608,...,0.584,0.655,0.761,0.365,0.565,0.394,0.038,0.240,0.869,0.431
GSM1343558.1,72.2,0.678,0.327,0.685,0.681,0.253,0.599,0.531,0.057,0.619,...,0.608,0.587,0.688,0.412,0.499,0.380,0.067,0.231,0.816,0.455
GSM1870847.1,76.3,0.644,0.320,0.645,0.683,0.281,0.596,0.547,0.066,0.673,...,0.584,0.673,0.638,0.435,0.615,0.431,0.092,0.252,0.861,0.455
GSM1498556.1,62.0,0.645,0.363,0.709,0.692,0.284,0.582,0.566,0.064,0.585,...,0.607,0.655,0.692,0.455,0.550,NaN,0.065,0.257,0.842,NaN


In [32]:
park_blood_unhealthy['status']=[1]*len(park_blood_unhealthy.index)
cols = list(park_blood_unhealthy.columns.values)
cols=cols[0:-1]
park_blood_unhealthy = park_blood_unhealthy[['status']+cols]
park_blood_unhealthy

,status,AGE,cg01074797,cg06784991,cg07181952,cg07418114,cg16290275,cg26671246,cg15121420,cg22158769,...,cg23078123,cg25256723,cg25410668,cg06240854,cg06639320,cg00329615,cg03607117,cg04474832,cg16932827,cg26614073
GSM3035963,1,78.0,0.528,0.274,0.624,0.685,0.257,0.535,0.569,0.008,...,0.443,0.722,0.948,0.475,0.589,0.322,0.086,0.176,0.772,0.394
GSM3035809,1,78.0,0.581,0.333,0.696,0.643,0.208,0.597,0.521,0.029,...,0.556,0.713,0.744,0.365,0.532,0.439,0.088,0.232,0.775,0.372
GSM3035847,1,75.0,0.532,0.242,0.673,0.639,0.184,0.551,0.486,0.045,...,0.542,0.619,0.733,0.392,0.455,0.346,0.043,0.191,0.661,0.435
GSM1870906,1,71.4,0.555,0.321,0.665,0.627,0.230,0.580,0.479,0.024,...,0.591,0.638,0.917,0.409,0.576,0.250,0.081,0.215,0.661,0.417
GSM1871363,1,76.2,0.640,0.334,0.626,0.664,0.222,0.598,0.509,0.113,...,0.589,0.784,0.723,0.404,0.617,0.405,0.139,0.224,0.840,0.500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM3035602.1,1,79.0,0.595,0.289,0.593,0.689,0.227,0.574,0.547,0.042,...,0.584,0.655,0.761,0.365,0.565,0.394,0.038,0.240,0.869,0.431
GSM1343558.1,1,72.2,0.678,0.327,0.685,0.681,0.253,0.599,0.531,0.057,...,0.608,0.587,0.688,0.412,0.499,0.380,0.067,0.231,0.816,0.455
GSM1870847.1,1,76.3,0.644,0.320,0.645,0.683,0.281,0.596,0.547,0.066,...,0.584,0.673,0.638,0.435,0.615,0.431,0.092,0.252,0.861,0.455
GSM1498556.1,1,62.0,0.645,0.363,0.709,0.692,0.284,0.582,0.566,0.064,...,0.607,0.655,0.692,0.455,0.550,NaN,0.065,0.257,0.842,NaN


In [33]:

col_a=list(alz_blood_unhealthy.columns[1:])
drop_a=[]
for i in col_a:
    if i not in comb:
        drop_a.append(i)

alz_blood_unhealthy=alz_blood_unhealthy.drop(columns=drop_a)    
alz_blood_unhealthy 



,AGE,cg01074797,cg06784991,cg07181952,cg07418114,cg16290275,cg26671246,cg15121420,cg22158769,cg22454769,...,cg23078123,cg25256723,cg25410668,cg06240854,cg06639320,cg00329615,cg03607117,cg04474832,cg16932827,cg26614073
GSM1069225,81.0,0.525,0.406,0.617,0.632,0.159,0.592,0.475,0.022,0.655,...,0.548,0.587,0.635,0.363,0.530,0.284,0.081,0.193,0.714,0.436
GSM1069226,88.0,0.582,0.336,0.612,0.693,0.084,0.509,0.526,0.077,0.705,...,0.574,0.610,0.746,0.386,0.606,NaN,0.117,0.158,0.760,0.419
GSM1069227,91.0,0.558,0.310,0.635,0.694,0.107,0.559,0.550,0.064,0.639,...,0.557,0.555,0.785,0.363,0.538,NaN,0.090,0.143,0.776,0.291
GSM1069228,87.0,0.583,0.378,0.678,0.681,0.180,0.586,0.573,0.039,0.684,...,0.526,0.631,0.688,0.438,0.559,0.409,0.052,0.227,0.795,0.482
GSM1069229,89.0,0.519,0.270,0.540,0.596,0.046,0.498,0.386,0.035,0.667,...,0.432,0.510,0.691,0.289,0.587,0.207,0.094,0.154,0.580,0.284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM1443699,88.0,0.450,0.252,0.623,0.668,0.157,0.560,0.552,0.044,0.560,...,0.596,0.610,0.646,0.343,0.504,0.398,0.085,0.230,0.733,0.371
GSM1443711,74.0,0.595,0.296,0.697,0.691,0.209,0.601,0.504,0.019,0.687,...,0.550,0.641,0.631,0.385,0.513,0.336,0.098,0.182,0.814,0.355
GSM1443721,74.0,0.555,0.367,0.613,0.677,0.186,0.538,0.462,0.029,0.605,...,0.583,0.588,0.637,0.393,0.526,0.339,0.106,0.219,0.751,0.418
GSM1443736,89.0,0.535,0.296,0.565,0.611,0.068,0.508,0.360,0.041,0.597,...,0.596,0.595,0.603,0.318,0.536,0.277,0.076,0.172,0.606,0.409


In [34]:
alz_blood_unhealthy['status']=[2]*len(alz_blood_unhealthy.index)
cols = list(alz_blood_unhealthy.columns.values)
cols=cols[0:-1]
alz_blood_unhealthy = alz_blood_unhealthy[['status']+cols]
alz_blood_unhealthy

,status,AGE,cg01074797,cg06784991,cg07181952,cg07418114,cg16290275,cg26671246,cg15121420,cg22158769,...,cg23078123,cg25256723,cg25410668,cg06240854,cg06639320,cg00329615,cg03607117,cg04474832,cg16932827,cg26614073
GSM1069225,2,81.0,0.525,0.406,0.617,0.632,0.159,0.592,0.475,0.022,...,0.548,0.587,0.635,0.363,0.530,0.284,0.081,0.193,0.714,0.436
GSM1069226,2,88.0,0.582,0.336,0.612,0.693,0.084,0.509,0.526,0.077,...,0.574,0.610,0.746,0.386,0.606,NaN,0.117,0.158,0.760,0.419
GSM1069227,2,91.0,0.558,0.310,0.635,0.694,0.107,0.559,0.550,0.064,...,0.557,0.555,0.785,0.363,0.538,NaN,0.090,0.143,0.776,0.291
GSM1069228,2,87.0,0.583,0.378,0.678,0.681,0.180,0.586,0.573,0.039,...,0.526,0.631,0.688,0.438,0.559,0.409,0.052,0.227,0.795,0.482
GSM1069229,2,89.0,0.519,0.270,0.540,0.596,0.046,0.498,0.386,0.035,...,0.432,0.510,0.691,0.289,0.587,0.207,0.094,0.154,0.580,0.284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM1443699,2,88.0,0.450,0.252,0.623,0.668,0.157,0.560,0.552,0.044,...,0.596,0.610,0.646,0.343,0.504,0.398,0.085,0.230,0.733,0.371
GSM1443711,2,74.0,0.595,0.296,0.697,0.691,0.209,0.601,0.504,0.019,...,0.550,0.641,0.631,0.385,0.513,0.336,0.098,0.182,0.814,0.355
GSM1443721,2,74.0,0.555,0.367,0.613,0.677,0.186,0.538,0.462,0.029,...,0.583,0.588,0.637,0.393,0.526,0.339,0.106,0.219,0.751,0.418
GSM1443736,2,89.0,0.535,0.296,0.565,0.611,0.068,0.508,0.360,0.041,...,0.596,0.595,0.603,0.318,0.536,0.277,0.076,0.172,0.606,0.409


In [35]:
frames=[healthy_df,park_blood_unhealthy,alz_blood_unhealthy]
#key=healthy_df.columns
result = pd.concat(frames)
result

sample_id,status,AGE,cg01074797,cg06784991,cg07181952,cg07418114,cg16290275,cg26671246,cg15121420,cg22158769,...,cg23078123,cg25256723,cg25410668,cg06240854,cg06639320,cg00329615,cg03607117,cg04474832,cg16932827,cg26614073
GSM2334366,0,94.0,0.688,0.457,0.688,0.713,0.232,0.598,0.565,0.096,...,0.465,0.805,0.597,0.474,0.594,0.357,0.194,0.300,0.835,0.287
GSM989882,0,96.0,0.504,0.316,0.602,0.664,0.193,0.582,0.429,0.070,...,0.558,0.576,0.699,0.391,0.704,0.377,0.078,0.223,0.659,0.592
GSM989863,0,101.0,0.571,0.435,0.673,0.634,0.148,0.557,0.442,0.047,...,0.471,0.554,0.652,0.236,0.619,0.327,0.096,0.190,0.650,0.506
GSM1443696,0,99.0,0.440,0.335,0.636,0.663,0.143,0.557,0.537,0.110,...,0.560,0.714,0.663,0.537,0.579,0.385,0.180,0.232,0.611,0.422
GSM1069241,0,99.0,0.440,0.335,0.636,0.663,0.143,0.557,0.537,0.110,...,0.560,0.714,0.663,0.537,0.579,0.385,0.180,0.232,0.611,0.422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM1443699,2,88.0,0.450,0.252,0.623,0.668,0.157,0.560,0.552,0.044,...,0.596,0.610,0.646,0.343,0.504,0.398,0.085,0.230,0.733,0.371
GSM1443711,2,74.0,0.595,0.296,0.697,0.691,0.209,0.601,0.504,0.019,...,0.550,0.641,0.631,0.385,0.513,0.336,0.098,0.182,0.814,0.355
GSM1443721,2,74.0,0.555,0.367,0.613,0.677,0.186,0.538,0.462,0.029,...,0.583,0.588,0.637,0.393,0.526,0.339,0.106,0.219,0.751,0.418
GSM1443736,2,89.0,0.535,0.296,0.565,0.611,0.068,0.508,0.360,0.041,...,0.596,0.595,0.603,0.318,0.536,0.277,0.076,0.172,0.606,0.409


In [36]:
result.to_csv('combined_healthy_unhealthy_77_cpgs.csv', encoding='utf-8', index=True)

# Processing combined data

In [37]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score


In [38]:
combined_df = pd.read_csv('combined_healthy_unhealthy_77_cpgs.csv', index_col=0,low_memory=False)
combined_df

,status,AGE,cg01074797,cg06784991,cg07181952,cg07418114,cg16290275,cg26671246,cg15121420,cg22158769,...,cg23078123,cg25256723,cg25410668,cg06240854,cg06639320,cg00329615,cg03607117,cg04474832,cg16932827,cg26614073
GSM2334366,0,94.0,0.688,0.457,0.688,0.713,0.232,0.598,0.565,0.096,...,0.465,0.805,0.597,0.474,0.594,0.357,0.194,0.300,0.835,0.287
GSM989882,0,96.0,0.504,0.316,0.602,0.664,0.193,0.582,0.429,0.070,...,0.558,0.576,0.699,0.391,0.704,0.377,0.078,0.223,0.659,0.592
GSM989863,0,101.0,0.571,0.435,0.673,0.634,0.148,0.557,0.442,0.047,...,0.471,0.554,0.652,0.236,0.619,0.327,0.096,0.190,0.650,0.506
GSM1443696,0,99.0,0.440,0.335,0.636,0.663,0.143,0.557,0.537,0.110,...,0.560,0.714,0.663,0.537,0.579,0.385,0.180,0.232,0.611,0.422
GSM1069241,0,99.0,0.440,0.335,0.636,0.663,0.143,0.557,0.537,0.110,...,0.560,0.714,0.663,0.537,0.579,0.385,0.180,0.232,0.611,0.422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM1443699,2,88.0,0.450,0.252,0.623,0.668,0.157,0.560,0.552,0.044,...,0.596,0.610,0.646,0.343,0.504,0.398,0.085,0.230,0.733,0.371
GSM1443711,2,74.0,0.595,0.296,0.697,0.691,0.209,0.601,0.504,0.019,...,0.550,0.641,0.631,0.385,0.513,0.336,0.098,0.182,0.814,0.355
GSM1443721,2,74.0,0.555,0.367,0.613,0.677,0.186,0.538,0.462,0.029,...,0.583,0.588,0.637,0.393,0.526,0.339,0.106,0.219,0.751,0.418
GSM1443736,2,89.0,0.535,0.296,0.565,0.611,0.068,0.508,0.360,0.041,...,0.596,0.595,0.603,0.318,0.536,0.277,0.076,0.172,0.606,0.409


In [39]:
def mean_impute(data):
    nas=data.isnull().sum()    
    col_means=data.mean(axis=0)
    na_cols=[]
    na_cols_means=[]

    for i in range(len(nas)):
        if nas[i]!=0:        
            na_cols.append(nas.keys()[i])
            na_cols_means.append(col_means[i])
        
    ids=list(data.index)
    for i in ids:
        for j in range(len(na_cols)):
            if str(data.loc[i][na_cols[j]])=="nan":
                data.loc[i][na_cols[j]]=na_cols_means[j]
    
    
    return data

# ID appears in healthy and unhealthy cohorts

### Remove ages <20

In [40]:
combined_df=combined_df.loc[combined_df['AGE'] >= 20]
combined_df

,status,AGE,cg01074797,cg06784991,cg07181952,cg07418114,cg16290275,cg26671246,cg15121420,cg22158769,...,cg23078123,cg25256723,cg25410668,cg06240854,cg06639320,cg00329615,cg03607117,cg04474832,cg16932827,cg26614073
GSM2334366,0,94.0,0.688,0.457,0.688,0.713,0.232,0.598,0.565,0.096,...,0.465,0.805,0.597,0.474,0.594,0.357,0.194,0.300,0.835,0.287
GSM989882,0,96.0,0.504,0.316,0.602,0.664,0.193,0.582,0.429,0.070,...,0.558,0.576,0.699,0.391,0.704,0.377,0.078,0.223,0.659,0.592
GSM989863,0,101.0,0.571,0.435,0.673,0.634,0.148,0.557,0.442,0.047,...,0.471,0.554,0.652,0.236,0.619,0.327,0.096,0.190,0.650,0.506
GSM1443696,0,99.0,0.440,0.335,0.636,0.663,0.143,0.557,0.537,0.110,...,0.560,0.714,0.663,0.537,0.579,0.385,0.180,0.232,0.611,0.422
GSM1069241,0,99.0,0.440,0.335,0.636,0.663,0.143,0.557,0.537,0.110,...,0.560,0.714,0.663,0.537,0.579,0.385,0.180,0.232,0.611,0.422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM1443699,2,88.0,0.450,0.252,0.623,0.668,0.157,0.560,0.552,0.044,...,0.596,0.610,0.646,0.343,0.504,0.398,0.085,0.230,0.733,0.371
GSM1443711,2,74.0,0.595,0.296,0.697,0.691,0.209,0.601,0.504,0.019,...,0.550,0.641,0.631,0.385,0.513,0.336,0.098,0.182,0.814,0.355
GSM1443721,2,74.0,0.555,0.367,0.613,0.677,0.186,0.538,0.462,0.029,...,0.583,0.588,0.637,0.393,0.526,0.339,0.106,0.219,0.751,0.418
GSM1443736,2,89.0,0.535,0.296,0.565,0.611,0.068,0.508,0.360,0.041,...,0.596,0.595,0.603,0.318,0.536,0.277,0.076,0.172,0.606,0.409


In [69]:
#Validation - held out data
X = combined_df.iloc[:,2:]
y = combined_df.iloc[:,0]
X_work, X_hold, y_work, y_hold = train_test_split(X, y, test_size=0.15,random_state=7)

In [70]:
#result_final=mean_impute(result)
X_tr, X_tst, y_train, y_test = train_test_split(X_work, y_work, test_size=0.25,random_state=4)

In [71]:
X_train = mean_impute(X_tr)
X_test = mean_impute(X_tst)

In [72]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train,y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

In [47]:
def class_accuracy(true,pred):
    correct=0
    total=len(pred)
    for i in range(total):
        if pred[i]==true[i]:
            correct+=1
    return correct/total

In [73]:
train_acc = class_accuracy(y_train,y_train_pred)
test_acc = class_accuracy(y_test,y_test_pred)

In [74]:
print(train_acc,test_acc)

0.8398926654740608 0.7935656836461126


In [75]:
X_valid = mean_impute(X_hold)
y_valid_pred = model.predict(X_valid)

In [76]:
valid_acc = class_accuracy(y_hold,y_valid_pred)
print(valid_acc)

0.7992424242424242


In [78]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_test_pred,target_names=['Healthy','Parkinsons','Alzheimers']))

              precision    recall  f1-score   support

     Healthy       0.80      0.99      0.89       291
  Parkinsons       0.43      0.05      0.10        56
  Alzheimers       0.80      0.15      0.26        26

    accuracy                           0.79       373
   macro avg       0.68      0.40      0.41       373
weighted avg       0.74      0.79      0.72       373



In [79]:
print(classification_report(y_hold,y_valid_pred,target_names=['Healthy','Parkinsons','Alzheimers']))

              precision    recall  f1-score   support

     Healthy       0.80      0.99      0.89       207
  Parkinsons       0.50      0.05      0.10        38
  Alzheimers       1.00      0.21      0.35        19

    accuracy                           0.80       264
   macro avg       0.77      0.42      0.44       264
weighted avg       0.77      0.80      0.73       264



In [107]:
y_hold.astype(str).value_counts()

0    207
1     38
2     19
Name: status, dtype: int64

In [120]:
frames_reduces=[healthy_df.sample(n=300, random_state=2),park_blood_unhealthy,alz_blood_unhealthy]
#key=healthy_df.columns
result_reduced = pd.concat(frames_reduces)
result_reduced=result_reduced.loc[result_reduced['AGE'] >= 20]
result_reduced

sample_id,status,AGE,cg01074797,cg06784991,cg07181952,cg07418114,cg16290275,cg26671246,cg15121420,cg22158769,...,cg23078123,cg25256723,cg25410668,cg06240854,cg06639320,cg00329615,cg03607117,cg04474832,cg16932827,cg26614073
GSM2844249,0,60.0,0.628,0.274,0.755,0.729,0.157,0.644,0.639,0.043,...,0.562,0.655,0.477,0.374,0.374,0.410,0.061,0.228,0.884,0.480
GSM990406,0,85.0,0.537,0.287,0.674,0.687,0.151,0.593,0.544,0.062,...,0.551,0.597,0.595,0.295,0.454,0.330,0.087,0.215,0.776,0.451
GSM1343074,0,64.4,0.660,0.276,0.727,0.708,0.203,0.637,0.581,0.053,...,0.551,0.587,0.620,0.420,0.503,0.318,0.061,0.197,0.723,0.383
GSM1870805,0,88.7,0.472,0.272,0.576,0.608,0.178,0.559,0.476,0.056,...,0.651,0.437,0.704,0.401,0.529,0.379,0.044,0.182,0.616,0.343
GSM2334482,0,54.0,0.432,0.265,0.704,0.691,0.244,0.595,0.606,0.036,...,0.656,0.713,0.633,0.473,0.496,0.432,0.095,0.260,0.851,0.413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM1443699,2,88.0,0.450,0.252,0.623,0.668,0.157,0.560,0.552,0.044,...,0.596,0.610,0.646,0.343,0.504,0.398,0.085,0.230,0.733,0.371
GSM1443711,2,74.0,0.595,0.296,0.697,0.691,0.209,0.601,0.504,0.019,...,0.550,0.641,0.631,0.385,0.513,0.336,0.098,0.182,0.814,0.355
GSM1443721,2,74.0,0.555,0.367,0.613,0.677,0.186,0.538,0.462,0.029,...,0.583,0.588,0.637,0.393,0.526,0.339,0.106,0.219,0.751,0.418
GSM1443736,2,89.0,0.535,0.296,0.565,0.611,0.068,0.508,0.360,0.041,...,0.596,0.595,0.603,0.318,0.536,0.277,0.076,0.172,0.606,0.409


In [121]:
#Validation - held out data
X = result_reduced.iloc[:,2:]
y = result_reduced.iloc[:,0]
X_work, X_hold, y_work, y_hold = train_test_split(X, y, test_size=0.15,random_state=8)

In [122]:
#result_final=mean_impute(result)
X_tr, X_tst, y_train, y_test = train_test_split(X_work, y_work, test_size=0.25,random_state=4)
X_train = mean_impute(X_tr)
X_test = mean_impute(X_tst)
model = LogisticRegression(max_iter=1000)
model.fit(X_train,y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

In [123]:
train_acc = class_accuracy(y_train,y_train_pred)
test_acc = class_accuracy(y_test,y_test_pred)
print(train_acc,test_acc)

0.7671232876712328 0.7540983606557377


In [124]:
X_valid = mean_impute(X_hold)
y_valid_pred = model.predict(X_valid)

In [125]:
valid_acc = class_accuracy(y_hold,y_valid_pred)
print(valid_acc)

0.7471264367816092


In [126]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_test_pred,target_names=['Healthy','Parkinsons','Alzheimers']))
print(classification_report(y_hold,y_valid_pred,target_names=['Healthy','Parkinsons','Alzheimers']))

              precision    recall  f1-score   support

     Healthy       0.73      0.78      0.76        46
  Parkinsons       0.73      0.77      0.75        48
  Alzheimers       0.86      0.68      0.76        28

    accuracy                           0.75       122
   macro avg       0.77      0.74      0.76       122
weighted avg       0.76      0.75      0.75       122

              precision    recall  f1-score   support

     Healthy       0.79      0.77      0.78        43
  Parkinsons       0.68      0.88      0.77        32
  Alzheimers       1.00      0.33      0.50        12

    accuracy                           0.75        87
   macro avg       0.82      0.66      0.68        87
weighted avg       0.78      0.75      0.73        87



In [7]:
def read_by_cpg_list(df_loc,cpg_list):
    df=pd.read_csv(df_loc, index_col=0,low_memory=False)
    col_p=list(df.columns[1:])
    drop_p=[]
    for i in col_p:
        if i not in cpg_list:
            drop_p.append(i)

    df=df.drop(columns=drop_p)    
    return df

In [8]:
with open(r"cpgs_XGboost_brain_ranked", "rb") as input_file:
    xgb_hc_brain_cpgs = cPickle.load(input_file)
xgb_hc_brain_cpgs_100=list(xgb_hc_brain_cpgs)[0:100]

In [9]:
alz_brain_unhealthy=read_by_cpg_list('s3://ewasunhealthy/data/brain/alzheimers/alz_brain_unhealthy_all.csv',xgb_hc_brain_cpgs_100)

In [10]:
alz_brain_unhealthy

,AGE,cg00807959,cg01066472,cg13806070,cg15907146,cg17104258,cg24441324,cg22454769,cg23606718,cg24079702,...,cg23595055,ch.1.169608016R,cg04739123,cg16367511,cg18008766,cg19451698,cg04834794,cg07303143,cg21182694,cg23352942
GSM1068826,88.0,0.052,0.512,0.113,0.679,0.010,0.846,0.275,0.078,0.198,...,0.972,0.030,0.105,0.044,0.135,0.120,0.029,0.158,0.243,0.407
GSM1068827,92.0,0.103,0.519,0.142,0.532,0.015,0.811,0.275,0.030,0.187,...,0.959,0.037,0.115,0.125,0.172,0.136,0.052,0.106,0.265,0.444
GSM1068829,93.0,0.064,0.620,0.154,0.535,0.024,0.819,0.307,0.065,0.209,...,0.968,0.032,0.053,0.193,0.229,0.161,0.032,NaN,0.329,0.480
GSM1068832,96.0,0.049,0.549,0.133,0.575,0.019,0.871,0.289,0.028,0.189,...,0.980,0.026,0.058,0.290,0.162,0.089,0.024,NaN,0.317,0.422
GSM1068833,86.0,0.081,0.287,0.089,0.568,0.021,0.863,0.165,0.044,0.139,...,0.981,0.041,0.062,0.189,0.116,0.083,0.022,0.107,0.236,0.431
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM2809060,80.0,0.364,0.635,0.144,0.692,0.189,0.910,0.407,0.202,0.274,...,0.950,0.058,0.069,0.153,0.111,0.083,0.081,NaN,0.184,0.589
GSM2809061,70.0,0.366,0.520,0.118,0.554,0.394,0.887,0.383,0.221,0.249,...,0.910,0.051,0.075,0.117,0.090,0.058,0.054,NaN,0.214,0.592
GSM2809062,75.0,0.328,0.321,0.124,0.610,0.134,0.892,0.400,0.130,0.253,...,0.943,0.048,0.031,0.161,0.072,0.056,0.064,0.285,0.209,0.610
GSM2809063,79.0,0.356,0.537,0.145,0.668,0.143,0.895,0.390,0.166,0.251,...,0.935,0.055,0.051,0.117,0.110,0.054,0.074,NaN,0.220,0.540


In [11]:
alz_brain_unhealthy.to_csv('alz_brain_top_56.csv', encoding='utf-8', index=True)

In [12]:
hunt_brain_unhealthy=read_by_cpg_list('s3://ewasunhealthy/data/brain/huntingtons/hunt_brain_unhealthy_all.csv',xgb_hc_brain_cpgs_100)


In [13]:
hunt_brain_unhealthy

,AGE,cg00807959,cg01066472,cg13806070,cg15907146,cg17104258,cg24441324,cg22454769,cg23606718,cg24079702,...,cg23595055,ch.1.169608016R,cg04739123,cg16367511,cg18008766,cg19451698,cg04834794,cg07303143,cg21182694,cg23352942
GSM1871451,83.0,0.255,0.432,0.147,0.640,0.179,0.926,0.426,0.159,0.308,...,0.981,0.046,0.027,0.166,0.082,0.096,0.078,0.270,0.221,0.571
GSM1871457,48.0,0.257,0.554,0.032,0.648,0.173,0.909,0.311,0.112,0.190,...,0.968,0.036,0.050,0.126,0.047,0.048,0.030,0.204,0.227,0.571
GSM1871498,51.0,0.282,0.248,0.143,0.550,0.244,0.920,0.332,0.168,0.245,...,0.914,0.051,0.022,0.170,0.057,0.047,0.049,0.306,0.239,0.603
GSM1871545,65.0,0.335,0.470,0.114,0.617,0.178,0.888,0.352,0.178,0.243,...,0.954,0.048,0.080,0.177,0.063,0.077,0.051,0.333,0.311,0.597
GSM1871590,67.0,0.316,0.586,0.056,0.595,0.189,0.881,0.402,0.160,0.245,...,0.964,0.043,0.037,0.189,0.051,0.056,0.040,0.241,0.186,0.575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM1871815,56.0,0.231,0.244,0.117,0.587,0.175,0.919,0.265,0.140,0.173,...,0.959,0.066,0.048,0.112,0.052,0.038,0.025,0.161,0.181,0.425
GSM1871849,62.0,0.212,0.433,0.125,0.667,0.044,0.845,0.155,0.147,0.215,...,0.969,0.061,0.049,0.141,0.066,0.044,0.063,0.205,0.211,0.420
GSM1871852,58.0,0.250,0.449,0.120,0.623,0.212,0.917,0.273,0.130,0.182,...,0.935,0.067,0.035,0.141,0.069,0.048,0.043,0.233,0.176,0.496
GSM1871860,91.0,0.287,0.657,0.129,0.649,0.017,0.902,0.324,0.176,0.238,...,0.983,0.048,0.052,0.154,0.069,0.060,0.057,0.276,0.237,0.384


In [14]:
hunt_brain_unhealthy.to_csv('hunt_brain_top_56.csv', encoding='utf-8', index=True)

### Which features to use:

- Depends on whether the goal is healthy vs unhealthy or by category?

1. Top 100 from each of the 3 groups
2. Features with the greatest discrepensies between healthy and unhealthy
3. 